**Projeto LLM - RAG para documento PDF**
---

**Equipe**:

*   Álysson Soares (ass5@cesar.school)
*   Gabriel Leite (gpl@cesar.school)

# Instalações & Importações

In [ ]:
!pip install langchain-text-splitters
!pip install langchain-community
!pip install langchain-openai
!pip install langchain-core
!pip install faiss-cpu
!pip install PyPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling op

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
import requests
import getpass
import os

# Desenvolvimento

## Configurando API_KEY

Para utilizar os serviços da OpenAI, será necessário ativar a API_KEY e inserir no código abaixo

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Insira sua chave OpenAI API: ")

Insira sua chave OpenAI API: ··········


## Carregando Documento

Foi utilizada a biblioteca PyPDFLoader para ler diretamente o edital do concurso do Dataprev, a leitura foi possivel devido a formatação correta do PDF. Caso o PDF não esteja na formatação correta, teriamos que rodar algum algoritimo de OCR para capturar o texto.

In [ ]:
url = "https://github.com/GabrielPLeite/Topicos-Contemporaneos/raw/main/Projeto%20Final/dataprev_edital_0.pdf"
response = requests.get(url)

with open("dataprev_edital_0.pdf", "wb") as file:
    file.write(response.content)

print("Download concluído!")

Download concluído!


In [ ]:
# Atualizar para puxar diretamente do github
pdf_path = "/content/dataprev_edital_0.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

## Dividindo Documentos - Splitting/Chunking

Para preparar os textos para o processamento, foram criados chunks com 1000 caracteres, herdando 200 do anterior para manter o sentido.

In [ ]:
# Dividir os documentos em chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)

## Indexando - Store

A partir dos chunks criados, foi aplicado o embedding com FAISS para retornar informações através da similaridade.

In [ ]:
# Criar embeddings e indexar no FAISS
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

## Buscando e Recuperando Informações - Retrieve

Foi criado o template com as intruções para configurar o comportamento do assistente.

In [ ]:
system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

Usando o modelo "gpt-4", foi criado o sistema para receber os textos e responder as perguntas a partir dele.

In [ ]:
# Definição do modelo
llm = ChatOpenAI(model="gpt-4")

# Construir a pipeline RAG
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

## Gerando Respostas - Generate

In [ ]:
question = "Quantas vagas existem nesse concurso?"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: Existem 236 vagas nesse concurso.


In [ ]:
question = "Dentre as vagas, quantas são para pessoas com deficiência?"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: Ficam reservadas aos candidatos com deficiência, no mínimo, 5% (cinco por cento) do total das vagas que forem providas durante o prazo de validade do Concurso.


In [ ]:
question = "Qual é a data da prova?"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: A prova objetiva de múltipla escolha será realizada no dia 17 de novembro de 2024.


In [ ]:
question = "Para o cargo de analista de TI, com perfil de analise de negócios de ti, para a o estado de brasilia, existem quantas vagas?"
response = rag_chain.invoke(question)
print("Resposta:", response)

Resposta: Para o cargo de analista de TI, com perfil de análise de negócios de TI, em Brasília, existem 50 vagas.


In [ ]:
# Teste
question = input("Digite sua pergunta: ")
response = rag_chain.invoke(question)
print(" ")
print("Resposta:", response)